# Prune the AR6 database climate dump

All we want is the 1202 vetted scenarios from FaIR v1.6.2 with
- total anthro forcing (50th percentile)
- CO2 forcing (50th percentile)
- CH4 forcing (50th percentile)
- N2O forcing (50th percentile)

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
ar6_df = pd.read_csv('../scenario_data/1668008030411-AR6_Scenarios_Database_World_ALL_CLIMATE_v1/AR6_Scenarios_Database_World_ALL_CLIMATE_v1.1.csv')

In [ ]:
ar6_df

In [ ]:
xl = pd.ExcelFile('../scenario_data/1668008030411-AR6_Scenarios_Database_World_ALL_CLIMATE_v1/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx')
xl.sheet_names

In [ ]:
meta_df = xl.parse('meta')

In [ ]:
wanted_vars = [
    'AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|CO2|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|CH4|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|N2O|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Harmonized|Emissions|CO2|Energy and Industrial Processes',
    'AR6 climate diagnostics|Effective Radiative Forcing|FaIRv1.6.2|50.0th Percentile',
]

In [ ]:
new_df = pd.DataFrame(columns = ar6_df.columns)
new_df

In [ ]:
for ms, _ in tqdm(meta_df[meta_df['Category'].isin([f'C{i}' for i in range(1, 9)])].groupby(["Model", "Scenario"])):
    model = ms[0]
    scenario = ms[1]
    #ar6_df.loc[(ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))]
    new_df = pd.concat(
        [
            new_df,
            ar6_df.loc[
                (ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))
            ]
        ]
    )

In [ ]:
#list(ar6_df.Variable.unique())

In [ ]:
new_df

In [ ]:
othererf_df = pd.DataFrame(columns = ar6_df.columns)

In [ ]:
for idx, (ms, _) in tqdm(enumerate(new_df.groupby(["Model", "Scenario"]))):
    model = ms[0]
    scenario = ms[1]
    othererf_df.loc[idx, 'Model'] = model
    othererf_df.loc[idx, 'Scenario'] = scenario
    othererf_df.loc[idx, 'Variable'] = 'AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile'
    othererf_df.loc[idx, 'Region'] = 'World'
    othererf_df.loc[idx, 'Unit'] = 'W/m^2'
    othererf_df.loc[idx, '1995':] = (
        new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|CO2|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|CH4|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|N2O|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values
    )
#     new_df = pd.concat(
#         [
#             new_df,
#             ar6_df.loc[
#                 (ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))
#             ]
#         ]

In [ ]:
new_df = pd.concat((othererf_df, new_df))

In [ ]:
new_df.sort_values(by=['Model', 'Scenario']).to_csv('../data/ar6_wg3_scenarios.csv', index=False)